# Sentiment Analyzer using NLP

In [1]:
import tweepy
import configparser
import pandas as pd

#### configs

In [2]:
api_key = 'eL2DeeOX1wX7HH0Ldu3Jy6qNT'
api_key_secret = '0QnVbYfZlYxzuJI4Mx9GDIEb8uiQJlLoMBCyDH4V1KvxeCt6tF'
access_token = '852228407140261893-0sCUBrO7a2lnroq6sFhcRqCnn3QdPsS'
access_token_secret = 'qcHOxjOfFmrBaTkNp0rFJLLCtfL0hSR2ytQGOWlUo05Zj'

#### authentication

In [3]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
Elon_musk_tweet = api.user_timeline(user_id = "Elon Musk", screen_name = "elonmusk", tweets_mode = "extended", include_rts = False, count = 20000)

Unexpected parameter: tweets_mode


In [5]:
data = []
for tweet in Elon_musk_tweet:
    data.append([tweet.created_at, tweet.text])

In [6]:
columns = ['Time', 'Tweet']
df = pd.DataFrame(data, columns=columns)
df.to_csv('tweets.csv')

#### Import all csv files for use

In [7]:
Tesla_data = pd.read_csv('C:/Users/personal/Documents/datasets/TSLA.csv')
Tweets = pd.read_csv('tweets.csv')
raw_text = Tweets["Tweet"].values.tolist()

#### convert raw text to lower case

In [8]:
lower_text = [] 
def to_lower_case(data):
    for words in raw_text:
        lower_text.append(str.lower(words))         

In [9]:
to_lower_case(raw_text)

#### tokenize list of words

I will perform an overview of what basic processes of NLP sentence analyzer is all about below before diving fully into 
the main processes in the second half of this notebook

In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk

In [11]:
sent_toke = []
for e in lower_text:
    e = sent_tokenize(e)
    sent_toke.append(e)

#### word tokenize list of words

In [12]:
import re

In [13]:
clean_text = [word_tokenize(i) for i in lower_text]

#### cleaning special characters using regex(regular expression)

In [14]:
clean_text_3 = []
for words in clean_text:
    clean = []
    for w in words:
        regex = re.sub(r'[^\w\s]', " ", w)
        if regex != " ":
            clean.append(regex)
        clean_text_3.append(clean)            

## NOTE:

From the above processes, the tweet column from the dataframe was turned into a list, making it almost tasking to check
the relationship with the date column from the dataframe also...and would still pose another difficulty if we want to compare
it with the date from the Stock data and do some meaningful analysis from it.
So, owing to this problem, i will perfom another set of Tokenisation on the column itself using the dataframe!
I will start the tokenisation directly from the tweets.csv

#### Tokenising the tweet from the tweet df

In [15]:
Tweets

,Unnamed: 0,Time,Tweet
0,0,2022-07-30 00:23:46+00:00,@HistoryInPics Great style
1,1,2022-07-30 00:19:27+00:00,@TeslaPodcast @SenateDems @Sen_JoeManchin @Sen...
2,2,2022-07-30 00:09:07+00:00,Heatwave in Shortville 🥵
3,3,2022-07-29 23:04:22+00:00,@ICannot_Enough @DirtyTesLa 10.13 is probably ...
4,4,2022-07-29 15:12:58+00:00,"@DirtyTesLa We’re working super hard on 10.13,..."
...,...,...,...
189,189,2022-07-12 04:55:53+00:00,@alexkehr @BillyM2k @levie I do recommend a lo...
190,190,2022-07-12 04:39:44+00:00,@levie And too many little red or green LED in...
191,191,2022-07-12 04:38:33+00:00,@BillyM2k @levie YouTube keeps playing me the ...
192,192,2022-07-12 04:33:59+00:00,@EvaFoxU 🤣


Iterate over the column to remove punctuation using the regex and apply function and then the 
stopwords respectively before going to the next step which would be tokenisation.

## What is Regex?

A Regular Expression (or Regex) is a pattern (or filter) that describes a set of strings that matches the pattern. 
In other words, a regex accepts a certain set of strings and rejects the rest.
Regular expressions are particularly useful for defining filters.
Regular expressions contain a series of characters that define a pattern of text to be matched—to make a filter more specialized, or general.
For example, the regular expression ^AL[.]* searches for all items beginning with AL.

In [16]:
def remove_hyper_strings(tweet):
    clean_tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    clean_tweet = re.sub("#[A-Za-z0-9_]+","", clean_tweet)

    return clean_tweet

In [17]:
Tweets["Tweet"] = Tweets["Tweet"].apply(remove_hyper_strings)

## Tokenization

When we deal with text, we need to break it down into smaller pieces for analysis. To 
do this, tokenization can be applied. Tokenization is the process of dividing text into 
a set of pieces, such as words or sentences. These pieces are called tokens. Depending 
on what we want to do, we can define our own methods to divide the text into many 
tokens. Let's look at how to tokenize the input text using NLTK

In [18]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

def tokenize_tweet(tweet):
    
    tweet_tokens = tokenizer.tokenize(tweet)
    
    return  tweet_tokens

In [19]:
Tweets["Tweet"] = Tweets["Tweet"].apply(tokenize_tweet)

## What are Stopwords?

Stop words are a set of commonly used words in any language.
For example, in English, “the”, “is” and “and”, would easily qualify as stop words.
In NLP and text mining applications, stop words are used to eliminate unimportant words,
allowing applications to focus on the important words instead.

#### removing stopwords

In [20]:
from nltk.corpus import stopwords
import string

stopwords_english = stopwords.words('english')

punctuation = string.punctuation

def remove_stopwords_punctuations(tweet_tokens):
    
    clean_tweets = []
    
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in punctuation):
            clean_tweets.append(word)
            
    return clean_tweets

In [21]:
Tweets["Tweet"] = Tweets["Tweet"].apply(remove_stopwords_punctuations)

## Stemming

Stemming is a technique used to extract the base form of the words by removing affixes from them.
It is just like cutting down the branches of a tree to its stems. For example, the stem of the words eating, eats, eaten is eat.
Search engines use stemming for indexing the words.
It is the process of converting words to its most general form

In [22]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def get_stem(tweets_clean):
    
    stem_tweets = []
    
    for word in tweets_clean:
        stem_word = stemmer.stem(word)
        stem_tweets.append(stem_word)
        
    return stem_tweets

In [23]:
Tweets["Tweet"] = Tweets["Tweet"].apply(get_stem)

## Lemmatiztion

What is meant by lemmatization?
Lemmatization is the grouping together of different forms of the same word.
In search queries, lemmatization allows end users to query any version of a base word and get relevant results.
In Lemmatization root word is called Lemma.
A lemma (plural lemmas or lemmata) is the canonical form, dictionary form, or citation form of a set of words.
For example, runs, running, ran are all forms of the word run, therefore run is the lemma of all these words.

In [24]:
from nltk.stem import WordNetLemmatizer
lemmatize = WordNetLemmatizer()

def lemmatize_tweet(tweet_lem):
    
    final_tweet = []
    
    for word in tweet_lem:
        lem_word = lemmatize.lemmatize(word)
        final_tweet.append(lem_word)
        
    return final_tweet

In [25]:
Tweets["Tweet"] = Tweets["Tweet"].apply(lemmatize_tweet)

# Combine Processes

Putting the above methods in a sequential method to ease future work when working with data

In [26]:
def process_tweet(tweet):
    
    processed_tweet = remove_hyper_strings(tweet)
    tweet_tokens = tokenize_tweet(processed_tweet)
    clean_tweets = remove_stopwords_punctuations(tweet_tokens)
    stem_tweets = get_stem(clean_tweets)
    final_tweet = lemmatize_tweet(stem_tweets)
    
    return final_tweet

In [27]:
tweet_example = 'you mean you' 

project_test = process_tweet(tweet_example)

project_test

['mean']

## Building a Sentiment Analyzer using Naive Bayes Classifier

Sentiment analysis is the process of determining the sentiment of a piece of text. 
For example, it can be used to determine whether a movie review is positive 
or negative.
We will use a Naive Bayes classifier to build this sentiment analyzer. First, extract all 
the unique words from the text. The NLTK classifier needs this data to be arranged 
in the form of a dictionary so that it can ingest it. Once the text data is divided into 
training and testing datasets, the Naive Bayes classifier will be trained to classify 
the reviews into positive and negative. Afterward, the top most informative words 
to indicate positive and negative reviews can be calculated and displayed.

In [28]:
from nltk.corpus import movie_reviews
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_acc

In [29]:
def extract_features(words):
     return dict([(word, True) for word in words])
    
def load_file():
    # Load the reviews from the corpus 
    fileids_pos = movie_reviews.fileids('pos')
    fileids_neg = movie_reviews.fileids('neg')
    
    # Extract the features from the reviews
    features_pos = [(extract_features(movie_reviews.words(fileids=[f])), 'Positive') for f in fileids_pos]
    features_neg = [(extract_features(movie_reviews.words(fileids=[f])), 'Negative') for f in fileids_neg]
    
    return(features_pos, features_neg)

In [31]:
features_pos, features_neg = load_file()

#### Define the split between training and testing. In this case, we will allocate 80% for training and 20% for testing:

In [32]:
threshold = 0.8
num_pos = int(threshold * len(features_pos))
num_neg = int(threshold * len(features_neg))

In [33]:
features_train = features_pos[:num_pos] + features_neg[:num_neg]
features_test = features_pos[num_pos:] + features_neg[num_neg:]

In [34]:
len(features_train)

1600

In [36]:
len(features_test)

400

Train a NaiveBayesClassifier using the training data and compute the accuracy 
using the inbuilt accuracy method available in NLTK:

In [37]:
clf = NaiveBayesClassifier.train(features_train)

Check for model accuracy

In [38]:
nltk_acc(clf, features_test)

0.735

In [39]:
nltk_acc(clf, features_train)

0.9825

# Test Model on Tweet Data

In [40]:
# First get dataframe column as a series
list_of_tweet = Tweets["Tweet"].tolist()

In [41]:
for review in list_of_tweet:
    review

In [42]:
# Compute Probability
probabilities = clf.prob_classify(extract_features(review))

In [43]:
predicted_sentiment = probabilities.max()

In [44]:
predicted_sentiment

'Positive'

In [45]:
round(probabilities.prob(predicted_sentiment), 2)

0.53